### [How to chain runnables](https://python.langchain.com/docs/how_to/sequence/)

In [20]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [21]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Model

In [22]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Cuentame un chiste de {topic}")

chain = prompt | model | StrOutputParser()

# chain.invoke({"topic": "programacion"})
chunks = []
for chunk in chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|¡|Claro|!| Aquí| tienes| uno|:

|¿Por| qué| los| program|adores| conf|unden| Halloween| y| Navidad|?

|¡|Porque| OCT| |31| es| igual| a| DEC| |25|!| 

|(|Exp|lic|ación|:| en| programación|,| "|O|CT|"| se| refiere| a| la| base| oct|al| (|base| |8|)| y| "|DEC|"| a| la| base| decimal| (|base| |10|).| En| base| oct|al|,| |31| es| igual| a| |3|*|8| +| |1| =| |25| en| decimal|).||

#### B. Coercion

In [24]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("Analiza el chiste: {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

# composed_chain.invoke({"topic": "programacion"})
chunks = []
for chunk in composed_chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|¡|Claro|!| Vamos| a| analizar| el| ch|iste|:

|1|.| **|Context|o| y| temática|**|:| El| ch|iste| se| sit|úa| en| el| mundo| de| la| programación|,| un| campo| muy| específico| que| generalmente| es| conocido| por| su| jer|ga| técnica| y| sus| propios| ch|istes| internos|.| El| humor| aquí| se| basa| en| la| cultura| de| los| program|adores| y| su| relación| con| el| café|.

|2|.| **|Juego| de| palabras|**|:| La| b|roma| se| basa| en| un| juego| de| palabras| entre| "|c|afé| oscuro|"| y| "|c|ódigo| claro|".| Por| un| lado|,| "|c|afé| oscuro|"| se| refiere| a| un| tipo| de| café|,| mientras| que| "|c|ódigo| claro|"| se| refiere| a| un| código| que| es| fácil| de| leer| y| entender|.| Este| juego| de| palabras| crea| una| conexión| humor|ística| entre| dos| conceptos| que|,| a| primera| vista|,| parecen| no| estar| relacionados|.

|3|.| **|La| resolución|**|:| La| segunda| parte| del| ch|iste|,| "|no| tiene| bugs|",| juega| con| la| idea| de| que| un| "|c|ódigo| claro|"| es| menos| prop|

#### C. Test

In [39]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Paso 1: Crear el primer chain para generar un chiste
joke_prompt = ChatPromptTemplate.from_template("Cuentame un chiste de {topic}")
joke_chain = joke_prompt | model | StrOutputParser()

# Paso 2: Crear el segundo chain para analizar el chiste
analysis_prompt = ChatPromptTemplate.from_template("Analiza el chiste: {joke}")
composed_chain = {"joke": joke_chain} | analysis_prompt | model | StrOutputParser()

# Paso 3: Realizar el streaming del composed_chain
chunks = []
for chunk in composed_chain.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)


|Este| ch|iste| se| bas|a| en| un| juego| de| palabras| y| conoc|imientos| de| sistemas| de| numer|ación| en| program|ación|.| En| program|ación|,| a| men|udo| se| us|an| diferentes| bases| de| numer|ación|,| no| solo| la| base| |10|,| que| us|amos| en| la| vida| cot|id|iana|.| La| base| |16| se| llama| hexadecimal| (|HE|X|),| y| la| base| |8| se| llama| oct|al| (|O|CT|).

|En| el| ch|iste|,| "|31| OCT|"| es| interpret|ado| como| un| número| en| base| |8| (|oct|al|),| mientras| que| "|25| DEC|"| es| interpret|ado| como| un| número| en| base| |10| (|decimal|,| que| es| el| sistema| que| us|amos| normal|mente|).| 

|El| número| |31| en| base| oct|al| es| igual| al| número| |25| en| base| decimal|.| Ent|onces|,| aunque| pare|z|ca| que| estamos| compar|ando| fe|chas| (|31| de| oct|ubre|,| que| es| Halloween|,| y| |25| de| dici|embre|,| que| es| Nav|idad|),| en| realidad| estamos| compar|ando| números| en| diferentes| bases| de| numer|ación|.| 

|Por| lo| tanto|,| el| ch|iste| es| una| espe

In [45]:
events = [
    event
    async for event in composed_chain.astream_events(
        {"topic": "programacion"},
        version="v2",
    )
]

In [46]:
events

[{'event': 'on_chain_start',
  'data': {'input': {'topic': 'programacion'}},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': 'cb03db45-2e0a-4222-927c-74b6b2a2cc7c',
  'metadata': {},
  'parent_ids': []},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableParallel<joke>',
  'tags': ['seq:step:1'],
  'run_id': '1cbbbb99-1f89-4e5d-a3a9-5c2aa0ec44f6',
  'metadata': {},
  'parent_ids': ['cb03db45-2e0a-4222-927c-74b6b2a2cc7c']},
 {'event': 'on_chain_start',
  'data': {},
  'name': 'RunnableSequence',
  'tags': ['map:key:joke'],
  'run_id': 'afaa9acb-f782-4bbc-ac61-6da0bc74c8d1',
  'metadata': {},
  'parent_ids': ['cb03db45-2e0a-4222-927c-74b6b2a2cc7c',
   '1cbbbb99-1f89-4e5d-a3a9-5c2aa0ec44f6']},
 {'event': 'on_prompt_start',
  'data': {'input': {'topic': 'programacion'}},
  'name': 'ChatPromptTemplate',
  'tags': ['seq:step:1'],
  'run_id': '20713c24-26c0-40b5-857e-491f306bfded',
  'metadata': {},
  'parent_ids': ['cb03db45-2e0a-4222-927c-74b6b2a2cc7c',
   '1cbbbb99-1f89-

In [47]:
num_events = 0

async for event in composed_chain.astream_events({"topic": "programacion"}, version="v2"):
    kind = event["event"]
    
    if kind == "on_chat_model_stream":
        chunk = event["data"]["chunk"].content
        print(f"Chat model chunk: {repr(chunk)}", flush=True)

    elif kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(f"Parser chunk: {chunk}", flush=True)

    elif kind == "on_chain_stream":
        chunk = event["data"].get("chunk", None)
        if isinstance(chunk, dict):
            for key, value in chunk.items():
                print(f"Chain {key} chunk: {repr(value)}", flush=True)
        else:
            print(f"Chain chunk: {repr(chunk)}", flush=True)

    num_events += 1

    if num_events > 30:  # Truncate output if too many events
        print("...", flush=True)
        break

Chat model chunk: ''
Parser chunk: 
Chain chunk: ''
Chain joke chunk: ''
Chat model chunk: '¿'
Parser chunk: ¿
Chain chunk: '¿'
Chain joke chunk: '¿'
Chat model chunk: 'Por'
Parser chunk: Por
Chain chunk: 'Por'
Chain joke chunk: 'Por'
Chat model chunk: ' qué'
Parser chunk:  qué
Chain chunk: ' qué'
Chain joke chunk: ' qué'
Chat model chunk: ' los'
Parser chunk:  los
Chain chunk: ' los'
Chain joke chunk: ' los'
Chat model chunk: ' program'
Parser chunk:  program
Chain chunk: ' program'
Chain joke chunk: ' program'
...


#### D. Metodo pipe

In [48]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

'Este chiste hace referencia a la serie de televisión "Battlestar Galactica", en la cual los Cylons son una raza de robots que buscan exterminar a la humanidad. La broma juega con la idea de que los Cylons son expertos en encontrar a los humanos, ya que tienen la capacidad de "ver" a través de las paredes con sus sensores avanzados. La ironía y el humor negro son elementos presentes en este chiste, ya que se trata de una situación de peligro y amenaza para los humanos, pero se aborda de manera humorística.'

In [50]:
composed_chain_with_pipe = RunnableParallel({"joke": chain}).pipe(
    analysis_prompt, model, StrOutputParser()
)

chunks = []
for chunk in composed_chain_with_pipe.stream({"topic": "programacion"}):
    chunks.append(chunk)
    print(chunk, end='|', flush=True)

|Este| ch|iste| jue|ga| con| el| do|ble| sentido| de| la| palabra| "|bugs|",| que| en| ingl|és| significa| tanto| "|in|sect|os|"| como| "|erro|res| en| un| programa| de| comput|adora|".| La| b|roma| se| bas|a| en| la| idea| de| que| los| program|adores| pref|ieren| el| café| al| té| porque| el| té| tiene| demasi|ados| "|bugs|"| (|erro|res|),| lo| cual| es| una| situación| que| los| program|adores| intent|an| evitar| en| su| trabajo|.| Es| un| ch|iste| s|encil|lo| pero| efect|ivo| para| sac|ar| una| son|ris|a| a| quienes| trabaj|an| en| el| campo| de| la| program|ación|.||